In [1]:
from textblob import TextBlob
import math
from collections import Counter, defaultdict
import tweepy as tw
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
api_key = "HAcIeikl6eRheQP1oKYGDwPx0"
api_secret = "gEWmqG7QVXzudnviUXDo98L2UKGYg9PNqy1bAR1geClkHjqBGk"
access_token = "1481095074465579008-1pApYlU4HRJWpbAXcbdVEVGzdACuEz"
access_token_secret = "htgD81H4mOxXw5MglAljFRHhcfGAuwsNomXVqvGw0Obwe"
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [3]:
number_of_tweets = 200
tweets = []
user_id =[]
created_at = []
keyword_list = ['News', 'Stock'] # , 'Customers', 'Employees', 'Company', 'Price', 'Shareholders', 'Market', 'Soaring', 'Trends'
company = 'Amazon'
final_kw_list = []

# Crawl data from keywords
for i in range (len(keyword_list)):
    kw = str(company + ' ' + keyword_list[i])
    print(kw)
    final_kw_list.append(kw)
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)

Amazon News
Amazon Stock


In [4]:
df = pd.DataFrame({"Tweets":tweets})
df

,Tweets
0,"RT @PATPmovie: We give you the news, just a ye..."
1,RT @phoenixrites: Amazon deforestation: Record...
2,RT @writingben: I'm still reeling from the new...
3,RT @SiobhanIsaac: Warm your day up with this s...
4,RT @ProfMarkMaslin: Amazon deforestation: Reco...
...,...
395,(US)In Stock: GIGABYTE AORUS RX 6900 XT Xtreme...
396,🛎️ IN STOCK ALERT 🛎️\n\nGeForce RTX 3080 Ti - ...
397,RT @StockMKTNewz: The biggest single day marke...
398,RT @StockMKTNewz: The biggest single day marke...


In [6]:
df = df[~df.Tweets.str.contains("RT")]
df = df.reset_index(drop = True)

In [7]:
df['clean_documents'] = df['Tweets'].str.replace("[^a-zA-Z#]", " ")#remove words have letters less than 3
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))#lowercase all characters
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: x.lower())
df

C:\Users\hlinh\AppData\Local\Temp/ipykernel_16672/1530896253.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_documents'] = df['Tweets'].str.replace("[^a-zA-Z#]", " ")#remove words have letters less than 3


,Tweets,clean_documents
0,AVAILABLE NOW!!!\n\n#News #Virgins #Billionair...,available now #news #virgins #billionaires #sa...
1,"Warm your day up with this steamy, fast-paced ...",warm your day with this steamy fast paced read...
2,Xbox Series S|X restock updates for retailers ...,xbox series restock updates for retailers incl...
3,TAKE #AliElAli don't wanna #live in #technolog...,take #alielali don wanna #live #technology #ai...
4,Although TikTok is fun and Amazon employees h...,although tiktok fun and amazon employees have ...
...,...,...
140,(US)In Stock: PowerColor Hellhound RX 6600 - 6...,stock powercolor hellhound atc https fwlcymy l...
141,Yes! Vinyl Back In Stock! https://t.co/njmguZN...,yes vinyl back stock https njmguznpp #amazon
142,Mid-year stock up on much need supplies. #clea...,mid year stock much need supplies #clearthelis...
143,(US)In Stock: GIGABYTE AORUS RX 6900 XT Xtreme...,stock gigabyte aorus xtreme waterforce atc htt...


In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')# tokenization
tokenized_doc = df['clean_documents'].fillna('').apply(lambda x: x.split())# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
df['clean_documents'] = detokenized_doc
df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hlinh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Tweets,clean_documents
0,AVAILABLE NOW!!!\n\n#News #Virgins #Billionair...,available #news #virgins #billionaires #safe #...
1,"Warm your day up with this steamy, fast-paced ...",warm day steamy fast paced read #eroticromance...
2,Xbox Series S|X restock updates for retailers ...,xbox series restock updates retailers includin...
3,TAKE #AliElAli don't wanna #live in #technolog...,take #alielali wanna #live #technology #ai #ne...
4,Although TikTok is fun and Amazon employees h...,although tiktok fun amazon employees great sen...
...,...,...
140,(US)In Stock: PowerColor Hellhound RX 6600 - 6...,stock powercolor hellhound atc https fwlcymy l...
141,Yes! Vinyl Back In Stock! https://t.co/njmguZN...,yes vinyl back stock https njmguznpp #amazon
142,Mid-year stock up on much need supplies. #clea...,mid year stock much need supplies #clearthelis...
143,(US)In Stock: GIGABYTE AORUS RX 6900 XT Xtreme...,stock gigabyte aorus xtreme waterforce atc htt...


In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer


sentiments = SentimentIntensityAnalyzer()
df['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in df["clean_documents"]]
score = df["Compound"].values
sentiment = []
for i in score:
    if i > 0 :
        sentiment.append(1)
    elif i < 0 :
        sentiment.append(0)
    else:
        sentiment.append('Neutral')
df["Sentiment"] = sentiment
df.drop(df.loc[df['Sentiment']=='Neutral'].index, inplace=True)
df = df.reset_index(drop=True)
df

,Tweets,clean_documents,Compound,Sentiment
0,"Warm your day up with this steamy, fast-paced ...",warm day steamy fast paced read #eroticromance...,0.2263,1
1,Xbox Series S|X restock updates for retailers ...,xbox series restock updates retailers includin...,0.7096,1
2,Although TikTok is fun and Amazon employees h...,although tiktok fun amazon employees great sen...,0.9045,1
3,Beyond your wildest dreams? https://t.co/OWfI...,beyond wildest dreams https owfiu wiau #firetv...,0.4019,1
4,Galaxy S22 screen protectors are here to cover...,galaxy screen protectors cover new precious ph...,0.9559,1
...,...,...,...,...
132,(US)In Stock: PowerColor Hellhound RX 6600 - 6...,stock powercolor hellhound atc https fwlcymy l...,0.1779,1
133,Yes! Vinyl Back In Stock! https://t.co/njmguZN...,yes vinyl back stock https njmguznpp #amazon,0.4019,1
134,Mid-year stock up on much need supplies. #clea...,mid year stock much need supplies #clearthelis...,0.7579,1
135,(US)In Stock: GIGABYTE AORUS RX 6900 XT Xtreme...,stock gigabyte aorus xtreme waterforce atc htt...,0.1779,1


In [12]:
# tokenization
tokenized_doc = df['clean_documents'].fillna('').apply(lambda x: x.split())

# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stopwords.words('english')])

# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['clean_tweets'] = detokenized_doc

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(df['clean_tweets'])

In [14]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=42)
lsa = svd_model.fit_transform(X)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
# X = df['clean_documents'].values
y = df['Sentiment'].values.astype('float')

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(lsa, y, random_state=42, test_size=0.2)
lreg = LogisticRegression()
lreg.fit(X_train_bow, y_train_bow) # training the model
prediction = lreg.predict_proba(X_test_bow) # predicting on the validation set
prediction_float = prediction[:,1] >= 0.3 # if prediction >= 0.2 then 1 or else 0
prediction_float = prediction_float.astype('float')
accuracy_score(y_test_bow, prediction_float)
# print(X_train_bow)

0.7142857142857143